## Imports

In [1]:
import pandas as pd
import numpy as np

In [2]:
from unidecode import unidecode
import re
import os
from countryinfo import CountryInfo
from googletrans import Translator

## Fuentes

A continuación se listan las url donde se obtienen los datos

 * Medallas ganadas por país

In [3]:
url_medallas = "https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table"

 * Lista de países por continente

In [4]:
url_paises = "https://es.wikipedia.org/wiki/Anexo:Pa%C3%ADses_por_continentes"

 * Equivalencias entre códigos DOI y ISO

In [5]:
url_codigos = "https://es.wikipedia.org/wiki/Anexo:Comparaci%C3%B3n_de_los_c%C3%B3digos_del_COI,_la_FIFA_y_la_ISO_3166"

Los dataframes generados se guardarán en el directorio llamado `data/`, bajo los siguientes nombres,
 * `info_countries.csv`: Información variada de los países
 * `olympics.csv`: Información de medallas ganadas por país
 * `olympics_excel.xlsx`: Información de medallas ganadas por país en formato excel

In [6]:
data_path = "data"
if not os.path.exists(data_path):
	os.mkdir(data_path)

## Funciones a utilizar

La siguiente función se usará para darle un formato más cómodo a las columnas del dataframe resultante

In [7]:
def tratamiento_columnas(dataframe):

	columns = dataframe.columns

	new_columns = []
	for col in columns:

		if not isinstance(col, str):
			new_columns.append(col)
			continue

		col = unidecode(col.lower()).strip()
		while "  " in col:
			col = col.replace("  ", " ")
		col = re.sub(r"\s", "_", col)

		if "superficie" in col:
			col = "superficie_km2"
		elif "poblacion" in col:
			col = "poblacion"
		elif "iso" in col:
			col = col.replace("iso", "ISO")
		elif "idh" in col:
			col = col.upper()
		elif col == "nombre_oficial":
			col = "pais"

		new_columns.append(col)

	dataframe.columns = new_columns

	return dataframe

## Extracción de los dataframes

#### Dataframe de medallas por país

A continuación se extrae la información que Wikipedia posee de las medallas olímpicas ganadas por país, tanto en olimpiadas de verano como en las de invierno.

Primero extraemos de la url correspondiente.

In [8]:
df = pd.read_html(url_medallas)[1]

Renombramos las columnas para que la manipulación del data frame sea más cómodo

In [9]:
columns = ['Country', 'Num_Summer', 'gold_summer', 'silver_summer',
       'bronze_summer', 'Total_summer', 'Num_Winter', 'gold_winter',
       'silver_winter', 'bronze_winter', 'Total_winter', 'Num_Games', 'gold',
       'silver', 'bronze', 'Combined_total']

df.columns = columns
df = df[df["Country"] != "Totals"]

Una columna posee tanto el nombre del país como el código COI, por lo que crearemos dos columnas más donde los separaremos

In [10]:
df["Country_code_coi"] = df.Country.str.extract(r"\((\w{3})\)")[0]
df["Country_name"] = df.Country.str.extract(r"(.*)\s\(")[0]
df["Country"] = df["Country_name"] + " (" + df["Country_code_coi"] + ")"
df = df[['Country', 'Country_name','Country_code_coi', 'Num_Summer', 'gold_summer', 'silver_summer',
       'bronze_summer', 'Total_summer', 'Num_Winter', 'gold_winter',
       'silver_winter', 'bronze_winter', 'Total_winter', 'Num_Games', 'gold',
       'silver', 'bronze', 'Combined_total']]

Las columnas del dataframe están listas y son las siguientes
 * `Country`: Nombre del país y código expedido por el Comité Olímpico Internacional.
 * `Country_name`: Nombre del país.
 * `Country_code_coi`: Código expedido por el Comité Olímpico Internacional.
 * `Num_Summer`: Número de participaciones en Olimpiadas de Verano.
 * `gold_summer`: Número de medallas de oro ganadas en Olimpiadas de Verano.
 * `silver_summer`: Número de medallas de plata ganadas en Olimpiadas de Verano.
 * `bronze_summer`: Número de medallas de bronce ganadas en Olimpiadas de Verano.
 * `Total_summer`: Total de medallas de ganadas en Olimpiadas de Verano.
 * `Num_Winter`: Número de participaciones en Olimpiadas de Invierno.
 * `gold_winter`: Número de medallas de oro ganadas en Olimpiadas de Invierno.
 * `silver_winter`: Número de medallas de plata ganadas en Olimpiadas de Invierno.
 * `bronze_winter`: Número de medallas de bronce ganadas en Olimpiadas de Invierno.
 * `Total_winter`: Total de medallas de ganadas en Olimpiadas de Invierno.
 * `Num_Games`: Total de participaciones.
 * `gold`: Total de medallas de oro ganadas.
 * `silver`: Total de medallas de plata ganadas.
 * `bronze`: Total de medallas de bronce ganadas.
 * `Combined_total`: Total de medallas combinadas.

El dataframe queda de la siguiente manera 

In [11]:
df.sample(10)

,Country,Country_name,Country_code_coi,Num_Summer,gold_summer,silver_summer,bronze_summer,Total_summer,Num_Winter,gold_winter,silver_winter,bronze_winter,Total_winter,Num_Games,gold,silver,bronze,Combined_total
78,Kuwait (KUW),Kuwait,KUW,14,0,0,3,3,0,0,0,0,0,14,0,0,3,3
50,United Team of Germany (EUA),United Team of Germany,EUA,3,28,54,36,118,3,8,6,5,19,6,36,60,41,137
62,Iceland (ISL),Iceland,ISL,22,0,2,2,4,19,0,0,0,0,41,0,2,2,4
87,Mexico (MEX),Mexico,MEX,25,13,27,37,77,10,0,0,0,0,35,13,27,37,77
25,Chile (CHI),Chile,CHI,25,3,8,4,15,18,0,0,0,0,43,3,8,4,15
141,Tonga (TGA),Tonga,TGA,11,0,1,0,1,2,0,0,0,0,13,0,1,0,1
41,Eritrea (ERI),Eritrea,ERI,7,0,0,1,1,2,0,0,0,0,9,0,0,1,1
107,Portugal (POR),Portugal,POR,26,6,11,15,32,9,0,0,0,0,35,6,11,15,32
74,Kenya (KEN),Kenya,KEN,16,39,44,41,124,4,0,0,0,0,20,39,44,41,124
85,Malaysia (MAS),Malaysia,MAS,15,0,8,7,15,2,0,0,0,0,17,0,8,7,15


In [12]:
df.shape

(162, 18)

#### Dataframe de paises

Con la finalidad de conocer un poco mejor a los países, vamos a importar un dataframe con un poco de su información.

Primero vamos con información relevante. Primer crearemos una función que permitirá hacer la extracción y el tratamiento correspondiente:

In [13]:
def validar_continente(pais, series):
	return any(map(lambda x: bool(re.search(pais, x)), series.values))

def titulos_tratamiento(string):
	pattern = re.compile(r"(.*?)(?:\[.+\])+.*(\(.+\))?")

	if (match_ := pattern.fullmatch(string)):
		return match_.group(1)
	else:
		return string

def data_frame_paises():
	mapping = {
		"Alemania" : "Europa",
		"China" : "Asia",
		"Angola" : "Africa",
		"Chile" : "América",
		"Australia" : "Oceanía"
	}

	df_continents = pd.read_html(url_paises)

	tablas = []

	df_tratados = map(tratamiento_columnas, df_continents)

	for tb in df_tratados:
		try:
			map_country = list(filter(lambda x: validar_continente(x, tb["pais"]), mapping.keys()))[0]
		except (KeyError, IndexError):
			continue

		tb["Continente"] = mapping[map_country]
		tablas.append(tb.copy())

	df_countries = pd.concat(tablas, axis=0)
	df_countries["pais"] = df_countries["pais"].map(titulos_tratamiento)

	return df_countries

def extraer_codigo_iso(series):
	traducidos = {
		'Botsuana': 'Botswana',
		'Comoras': 'Comoros',
		'Gambia': 'Republic of The Gambia',
		'Guinea-Bisáu': 'Guinea-Bissau',
		'Malaui': 'Malawi',
		'Mauricio': 'Mauritius',
		'República Democrática del Congo': 'Democratic Republic of the Congo',
		'República del Congo': 'Republic of the Congo',
		'Santo Tomé y Príncipe': 'São Tomé and Príncipe',
		'Somalilandia (República de Somalilandia)': 'Somaliland',
		'Suazilandia': 'Swaziland',  # Suazilandia cambió su nombre a Eswatini en 2018
		'Sudán del Sur (República de Sudán del Sur)': 'South Sudan',
		'Yibuti': 'Djibouti'
	}

	if isinstance(series["ISO_3166-1"], str):
		return series["ISO_3166-1"]

	t = Translator()
	pais_ = series.pais

	pais_ingles = t.translate(pais_, src="es", dest="en")
	countryinfo = CountryInfo(pais_ingles.text)

	try:
		return countryinfo.iso(3)
	except KeyError:
		countryinfo = CountryInfo(traducidos[pais_])
		try:
			return countryinfo.iso(3)
		except KeyError:
			return np.nan

In [14]:
df_countries = data_frame_paises()
df_countries = df_countries.dropna(axis=1, how="all")
df_countries = df_countries.rename(columns={"codigo_ISO_3166-1" : "ISO_3166-1"})
df_countries["ISO_3166-1"] = df_countries.apply(extraer_codigo_iso, axis=1)
df_countries = df_countries.dropna(subset="ISO_3166-1")
df_countries.sample(10)

,pais,ISO_3166-1,superficie_km2,poblacion,capital,idiomas_oficiales,moneda,Continente,idioma_oficial,pib_per_capita_(us$),IDH
48,Ucrania,UKR,603.550,44.209.733,Kiev,Ucraniano,"Grivna(₴, UAH).",Europa,NaN,NaN,NaN
6,Nueva Zelanda,NZL,270 534,3 939 000,Wellington,Inglés y Maori,dólar neozelandés,Oceanía,NaN,NaN,Alto
6,Brasil,BRA,8 514 877,205 823 665,Brasilia,Portugués,"Real(R$, BRL).",América,NaN,NaN,NaN
2,Benín,BEN,112 622,8 439 000,Porto Novo,NaN,Franco CFA de África Occidental,Africa,Francés,$1176,NaN
29,Luxemburgo,LUX,2.586,582.291,Ciudad de Luxemburgo,LuxemburguésFrancésAlemán,"Euro(€, EUR).",Europa,NaN,NaN,NaN
18,Guinea-Bisáu,GNB,36 125,1 586 000,Bisáu,NaN,Franco CFA de África Occidental,Africa,Portugués,$736,NaN
21,Lesoto,LSO,30 355,1 795 000,Maseru,NaN,Loti,Africa,SesotoInglés,$2113,NaN
46,Vietnam,VNM,331 210,95 261 021,Hanói,Vietnamita,"Đồng vietnamita(₫, VND)",Asia,NaN,NaN,NaN
36,República del Congo,COG,342 000,3 999 000,Brazzaville,NaN,Franco CFA de África Central,Africa,FrancésLingala,$1369,NaN
41,Tayikistán,TJK,144 100,8 330 946,Dusambé,Tayiko,Somoni tayiko(TJS),Asia,NaN,NaN,NaN


Dado que el código ISO y el código dado por el COI en ocasiones no coinciden, es necesario tener la comparación entre las dos, para eso usaremos otro dataframe

In [15]:
df_codigos = pd.read_html(url_codigos)[1]
df_codigos.columns = [
	"pais", "COI", "FIFA", "ISO", "Observaciones"
]
df_codigos["ISO"] = df_codigos["ISO"].dropna()
df_codigos

,pais,COI,FIFA,ISO,Observaciones
0,Afganistán,AFG,AFG,AFG,NaN
1,Åland,NaN,NaN,ALA,No es un país. Pertenece a Finlandia.
2,Albania,ALB,ALB,ALB,NaN
3,Alemania,GER,GER,DEU,NaN
4,Andorra,AND,AND,AND,NaN
...,...,...,...,...,...
249,Yemen,YEM,YEM,YEM,NaN
250,Yibuti,DJI,DJI,DJI,NaN
251,Wallis y Futuna,NaN,NaN,WLF,No es un país. Pertenece a Francia.
252,Zambia,ZAM,ZAM,ZMB,NaN


Solo queda unir ambos dataframes

In [16]:
df_countries.shape

(202, 11)

In [17]:
df_countries = pd.merge(df_countries, df_codigos[["COI", "FIFA", "ISO"]], how="left", left_on="ISO_3166-1", right_on="ISO").drop(columns=["ISO"])
df_countries = df_countries[['pais', 'ISO_3166-1', "COI", "FIFA", 'superficie_km2', 'poblacion', 'capital','idiomas_oficiales', 'moneda', 'Continente']]

In [18]:
df_countries.shape

(202, 10)

El dataframe de paises queda con las siguientes columnas
 * `pais`: Nombre de país en español.
 * `ISO_3166-1`: Código ISO 3166-1 alpha 3 dado al país.
 * `ISO`: 
 * `superficie_km2`: Extensión territorial dado en $m^2$.
 * `poblacion`: Número de habitantes.
 * `capital`: Capital administrativa del país.
 * `idiomas_oficiales`: Idiomas oficiales.
 * `moneda`: Moneda de curso legal.
 * `Continente`: Continente al que pertenecen: América, Asia, Africa, Europa, Oceania.
 * `IDH`: Índice de Desarrollo Humano.

In [19]:
df_countries.to_csv(os.path.join(data_path, "info_countries.csv"), index=False, encoding="utf-8")
df_countries.sample(10)

,pais,ISO_3166-1,COI,FIFA,superficie_km2,poblacion,capital,idiomas_oficiales,moneda,Continente
76,Kirguistán,KGZ,KGZ,KGZ,199 951,5 727 553,Biskek,KirguísRuso,Som kirguís(KGS).,Asia
22,Irlanda,IRL,IRL,IRL,70.273,4.952.473,Dublín,IrlandésInglés,"Euro(€, EUR).",Europa
131,Nigeria,NGA,NGR,NGA,923 768,133 530 000,Abuya,NaN,Naira,Africa
189,Fiyi,FJI,FIJ,FIJ,18 376,823 000,Suva,"Inglés, Fiyiano e Indostánico",Dólar fiyiano,Oceanía
174,Jamaica,JAM,JAM,JAM,10 991,2 970 340,Kingston,Inglés,"Dólar jamaiquino($, JMD).",América
15,España,ESP,ESP,ESP,505.370,48.563.476,Madrid,Castellano,"Euro(€, EUR).",Europa
100,Benín,BEN,BEN,BEN,112 622,8 439 000,Porto Novo,NaN,Franco CFA de África Occidental,Africa
144,Sudán,SDN,SUD,SDN,1 886 068,31 436 378,Jartum,NaN,Libra sudanesa,Africa
113,Gambia,GMB,GAM,GAM,10 380,1 517 000,Banjul,NaN,Dalasi,Africa
102,Burkina Faso,BFA,BUR,BFA,274 000,13 228 000,Uagadugú,NaN,Franco CFA de África Occidental,Africa


#### Creando dataframe de trabajo

Ahora vamos a complementar el dataframe de medallas con algunos datos de los países. Por supuesto, en las Olimpiadas no solo participan países, sino también algunas organizaciones, sin embargo, no los tomaremos en cuenta.

Comencemos con algo simple: agregando el código ISO y el continente al df de medallas

In [20]:
df = df.merge(df_countries[["ISO_3166-1", "COI", "Continente"]], how="left", left_on="Country_code_coi", right_on="COI").drop(columns=["COI"])
df = df[['Country', 'Country_name','Country_code_coi', "ISO_3166-1", "Continente", 'Num_Summer', 'gold_summer', 'silver_summer',
       'bronze_summer', 'Total_summer', 'Num_Winter', 'gold_winter',
       'silver_winter', 'bronze_winter', 'Total_winter', 'Num_Games', 'gold',
       'silver', 'bronze', 'Combined_total']]
df.sample(5)

,Country,Country_name,Country_code_coi,ISO_3166-1,Continente,Num_Summer,gold_summer,silver_summer,bronze_summer,Total_summer,Num_Winter,gold_winter,silver_winter,bronze_winter,Total_winter,Num_Games,gold,silver,bronze,Combined_total
71,Iraq (IRQ),Iraq,IRQ,IRQ,Asia,16,0,0,1,1,0,0,0,0,0,16,0,0,1,1
57,West Germany (FRG),West Germany,FRG,NaN,NaN,5,56,67,81,204,6,11,15,13,39,11,67,82,94,243
60,Greece (GRE),Greece,GRE,GRC,Europa,30,36,46,47,129,20,0,0,0,0,50,36,46,47,129
167,Individual Neutral Athletes (AIN),Individual Neutral Athletes,AIN,NaN,NaN,1,1,3,1,5,0,0,0,0,0,1,1,3,1,5
26,Cape Verde (CPV),Cape Verde,CPV,CPV,Africa,8,0,0,1,1,0,0,0,0,0,8,0,0,1,1


In [21]:
df.shape

(171, 20)

In [22]:
df.to_csv(os.path.join(data_path,"olympics.csv"), index=False, encoding="utf-8")
df.to_excel(os.path.join(data_path,"olympics_excel.xlsx"), index=False, encoding="utf-8")